Scraping data using the Python Reddit API wrapper

In [1]:
import praw

In [2]:
client_id = 'rWkFCWZXUQJ4Jg'
client_secret =  'sOlRFC9Y5W3UpOIOTZP0BySF-8Q'
user_agent = 'r/india flair'
username = 'Megs0804'
password = 'Mc123456@'

In [3]:
reddit = praw.Reddit(client_id = client_id, client_secret = client_secret, user_agent = user_agent, username = username, password = password)

In [4]:
subreddit = reddit.subreddit('India')

In [5]:
top = subreddit.top(limit = 1000)

In [6]:
topics_dict = {"title":[], "body":[], "flair":[]}

In [7]:
for submission in top:
    topics_dict["title"].append(submission.title)
    topics_dict["body"].append(submission.selftext)
    topics_dict["flair"].append(submission.link_flair_text)

In [8]:
import pandas as pd
topics_data = pd.DataFrame(topics_dict)

In [9]:
topics_data

,title,body,flair
0,Will donate thrice the number of upvotes (amou...,>**Note**: If you want to know what this is al...,[R]eddiquette
1,Indian reply to NYtimes cartoon on Paris clima...,,/r/all
2,"The essence of the Indian soap opera, distille...",,r/all
3,Fuck all Religion,"Fuck all religion. Fuck Hindusim, fuck Islam, ...",Politics
4,German exchange Student at IIT Madras is being...,,Politics
5,Tragedy of India,,r/all
6,Today's The Hindu,,Coronavirus
7,"If you are not moved by this picture, I wish I...",,Non-Political
8,Megathread: India-Pakistan border skirmish,There is a lot of news and speculation coming ...,[R]eddiquette
9,"""From midnight the entire country will go unde...",,Politics [Megathread]


In [10]:
topics_data.to_csv('reddit1.csv', index=False) 

In [11]:
topics_data['flair'].unique()

array(['[R]eddiquette', '/r/all', 'r/all', 'Politics', 'Coronavirus',
       'Non-Political', 'Politics [Megathread]', 'Policy/Economy',
       'Totally real', 'Business/Finance', 'Unverified',
       'Science/Technology', 'Sports', 'Photography', 'CAA-NRC',
       'Zoke Tyme', 'Official Sadness Thread', 'Demonetization', 'Food',
       'Politics -- Source in comments', 'Policy & Economy',
       'Policy/Economy -2017 Article ', 'AskIndia', 'Original Comics',
       'CAA-NRC-NPR', 'Misleading Headline ', 'OC', 'Misleading',
       'On Internet Shutdowns', 'Scheduled', 'Lifehacks', 'AMA',
       'Entertainment', None, '40 Martyrs', 'Goal Achieved!!!'],
      dtype=object)

Since PRAW caps the number of submissions scraped to just 1000, I attemped to use Google Scripts to scrape reddit data.
However, that did not work.

I then found out about the pushshift API and used the same to crawl 15000 submissions from r/india.

In [12]:
import requests

In [13]:
url = "https://api.pushshift.io/reddit/search/submission"
params = {"subreddit": "india"}
requests.get(url, params = {"subreddit": "india", "size": 0, "aggs" : "subreddit"}).json()["aggs"]

{'subreddit': [{'doc_count': 1049913, 'key': 'india'}]}

In [14]:
url = "https://api.pushshift.io/reddit/search/submission"

def crawl_page(subreddit: str, last_page = None):
    """Crawl a page of results from a given subreddit.

    :param subreddit: The subreddit to crawl.
    :param last_page: The last downloaded page.

    :return: A page or results.
    """
    params = {"subreddit": subreddit, "size": 500, "sort": "desc", "sort_type": "created_utc"}
    if last_page is not None:
        if len(last_page) > 0:
            # resume from where we left at the last page
            params["before"] = last_page[-1]["created_utc"]
        else:
          # the last page was empty, we are past the last page
          return []
    results = requests.get(url, params)
    if not results.ok:
        # something wrong happened
        raise Exception("Server returned status code {}".format(results.status_code))
    return results.json()["data"]

In [15]:
import time

def crawl_subreddit(subreddit, max_submissions = 15000):
    """
    Crawl submissions from a subreddit.

    :param subreddit: The subreddit to crawl.
    :param max_submissions: The maximum number of submissions to download.

    :return: A list of submissions.
    """
    submissions = []
    last_page = None
    while last_page != [] and len(submissions) < max_submissions:
        last_page = crawl_page(subreddit, last_page)
        submissions += last_page
        time.sleep(3)
    return submissions[:max_submissions]

In [16]:
latest_submissions = crawl_subreddit("india")

In [17]:
latest_submissions

[{'all_awardings': [],
  'allow_live_comments': False,
  'author': 'thakurketan007',
  'author_flair_css_class': None,
  'author_flair_richtext': [],
  'author_flair_text': None,
  'author_flair_type': 'text',
  'author_fullname': 't2_2vcei3rl',
  'author_patreon_flair': False,
  'author_premium': False,
  'awarders': [],
  'can_mod_post': False,
  'contest_mode': False,
  'created_utc': 1587640715,
  'domain': 'ragingraavan.blogspot.com',
  'full_link': 'https://www.reddit.com/r/india/comments/g6kuqx/review_on_hasmukh_netflix_series/',
  'gildings': {},
  'id': 'g6kuqx',
  'is_crosspostable': True,
  'is_meta': False,
  'is_original_content': False,
  'is_reddit_media_domain': False,
  'is_robot_indexable': True,
  'is_self': False,
  'is_video': False,
  'link_flair_background_color': '#5093d6',
  'link_flair_css_class': 'Non-Political',
  'link_flair_richtext': [{'e': 'text', 't': 'Non-Political'}],
  'link_flair_template_id': '8f105a84-7ea0-11e3-bba1-12313b0c91be',
  'link_flair_te

In [18]:
df = pd.DataFrame(latest_submissions)

In [23]:
list(df)

['all_awardings',
 'allow_live_comments',
 'author',
 'author_cakeday',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'banned_by',
 'can_mod_post',
 'contest_mode',
 'created_utc',
 'discussion_type',
 'distinguished',
 'domain',
 'edited',
 'full_link',
 'gilded',
 'gildings',
 'id',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_template_id',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media',
 'media_embed',
 'media_metadata',
 'media_only',
 'no_follow',
 'num_comments',
 'num_crossposts',
 'over_18',
 'parent_whitelist_status',
 'permalink',
 'pinned',
 '

In [30]:
(df['num_crossposts']==0).sum()

14939

I kept a few of the fields, which seemed like they could help determine the flair of a post.

In [32]:
data = df[['title', 'selftext', 'link_flair_text', 'num_comments', 'num_crossposts', 'score']].copy()

In [33]:
data

,title,selftext,link_flair_text,num_comments,num_crossposts,score
0,Review on hasmukh (netflix series ),,Non-Political,0,0,1
1,President approves ordinance making violence a...,,Non-Political,1,0,1
2,Amazon Delivering to more Pincodes,[removed],Coronavirus,0,0,1
3,Is Japan Ready For Long Battle Against COVID-19?,,Coronavirus,0,0,1
4,Chinese Communism Will Be Dealt By The Entire ...,,Coronavirus,0,0,1
5,Earth Day 2020: How a group of citizens revive...,,Non-Political,0,0,1
6,Why is she furious about her true identity tho?,,NaN,1,0,1
7,I know this question has been answered earlier...,[removed],AskIndia,0,0,1
8,वह मजबूर हैं भिकारी नही।,,Non-Political,0,0,1
9,"What does shana lala, mean?","Hello. Just watched the movie and heard "" mera...",Non-Political,1,0,1


In [34]:
data['link_flair_text'].unique()

array(['Non-Political', 'Coronavirus', nan, 'AskIndia', 'Photography',
       'Science/Technology', 'Politics', 'Policy/Economy',
       'Business/Finance', 'Scheduled', 'Sports', 'All CAPS.',
       '| Social Media Rules |',
       '| Unverified Content / Disreputed Source |', 'Food',
       'Low-effort self-post.', 'CAA-NRC-NPR',
       '| Not specific to India |', 'Low Quality/Non OC Meme',
       '| Repost |',
       '| Not Original/Relevant Title | | Social Media Rules |',
       '| Not in English |', 'Shitpost', '| Low-effort Self Post |',
       '| Not in English | | Unverified Content / Disreputed Source |',
       '| Not specific to India | | Stickied Topic |',
       '| Not Original/Relevant Title | | Custom (Informed OP) |',
       'Meta.', '| Custom (Informed OP) |',
       '| Not Original/Relevant Title |', '| Image Rule Violation |',
       '| Witch-hunting/Targeting User |', '| Stickied Topic |',
       'Dead Link',
       '| Not in English | | Social Media Rules | Post 

The resultant data had a lot of miscellaneous flair types with very few instances. I kept only those with more than 100 instances in the data.

In [35]:
data['link_flair_text'].value_counts()

Coronavirus                                                      5122
Non-Political                                                    2321
AskIndia                                                         1642
Politics                                                         1314
Science/Technology                                                410
Photography                                                       355
Policy/Economy                                                    342
Business/Finance                                                  318
Food                                                              137
Sports                                                            102
| Repost |                                                        100
| Not Original/Relevant Title |                                    82
Scheduled                                                          80
| Unverified Content / Disreputed Source |                         79
Low-effort self-post

In [38]:
freq = data['link_flair_text'].value_counts()
frequent_values = freq[freq > 100].index

dataR = data[data['link_flair_text'].isin(frequent_values)]

In [39]:
dataR['link_flair_text'].value_counts()

Coronavirus           5122
Non-Political         2321
AskIndia              1642
Politics              1314
Science/Technology     410
Photography            355
Policy/Economy         342
Business/Finance       318
Food                   137
Sports                 102
Name: link_flair_text, dtype: int64

In [40]:
dataR.to_csv('redditData.csv', index=False) 

In [45]:
import pickle
dict1 = pickle.load(open('dict.pkl','rb'))

In [47]:
len(dict1)

2128